In [8]:
%run -t numeric_mapping.py
data.head()

Hello World
Index(['Age', 'Race', 'Marital Status', 'T Stage', 'N Stage', '6th Stage',
       'differentiate', 'Grade', 'A Stage', 'Tumor Size', 'Estrogen Status',
       'Progesterone Status', 'Regional Node Examined',
       'Reginol Node Positive', 'Survival Months', 'Status'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   Age                     4024 non-null   int64
 1   T Stage                 4024 non-null   int64
 2   N Stage                 4024 non-null   int64
 3   6th Stage               4024 non-null   int64
 4   differentiate           4024 non-null   int64
 5   Grade                   4024 non-null   int64
 6   A Stage                 4024 non-null   int64
 7   Tumor Size              4024 non-null   int64
 8   Estrogen Status         4024 non-null   int64
 9   Progesterone Status    

,Age,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,1,1,4,1,3,0,4,1,1,24,1,60,1
1,50,2,2,7,2,2,0,35,1,1,14,5,62,1
2,58,3,3,9,2,2,0,63,1,1,14,7,75,1
3,58,1,1,4,1,3,0,18,1,1,2,1,84,1
4,47,2,1,5,1,3,0,41,1,1,3,1,50,1


In [9]:
#2 - Random Forest but regressive model. This might be better as we will be given an average umber rather than an estimated range.
#I would expect this to be less accurate as in right less often, but the mean squared error should be smaller.

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(42)

#Input parameters = everything except survival months and status
#re
x = data.drop(["Survival Months", "Status"], axis=1)
y = data["Survival Months"]

x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5)
# Use RandomForestRegressor for regression tasks
rf = RandomForestRegressor(random_state=42).fit(x_train, y_train)

# Making predictions based on the model
y_pred = rf.predict(x_test)
y_val_pred = rf.predict(x_val)


#Checking accuracy on validation set
mse_val = mean_squared_error(y_val, y_val_pred)
print("Mean Squared Error on Validation Set:", mse_val)

y_test_pred = rf.predict(x_test)
#Checking accuracy on test set
mse_test = mean_squared_error(y_test, y_test_pred)
print("Mean Squared Error on Test Set:", mse_test)


#Before removing race and marital status: 413.27
#After removing race and marital status: 413.27
#After removing status: 548.0517. Worse, but in our case all future participants are alive, so this is not a good metric to use.
#After including validation set: 531.53


Mean Squared Error on Validation Set: 531.5328174051976
Mean Squared Error on Test Set: 551.6731475119385


In [11]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
np.random.seed(42)

x = data.drop(["Survival Months", "Status"], axis=1)
y = data["Survival Months"]
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)


rf = RandomForestClassifier(random_state=0).fit(x_train, y_train)

y_pred = rf.predict(x_test)
y_val_pred = rf.predict(x_val)


# Checking the accuracy of the model
accuracy_test = accuracy_score(y_test, y_pred)
print("Accuracy on test only:", accuracy)

accuracy_val = accuracy_score(y_val, y_val_pred)
#Before removing race and maritial status: 0.0211
#After removing race and marital status: 0.0224
#After removing status: 0.01129. Worse, but in our case all future participants are alive, so this is not a good metric to use.


Accuracy on test only: 0.01240694789081886
